In [1]:
import weaviate

client = weaviate.connect_to_local()


In [2]:
COLBERT_CLASS_NAME="jqara_colbert"
MUVERA_CLASS_NAME = "jqara_muvera"

In [3]:
from weaviate.classes.config import Configure, Property, DataType

if client.collections.exists(COLBERT_CLASS_NAME):
    client.collections.delete(COLBERT_CLASS_NAME)

client.collections.create(
    name=COLBERT_CLASS_NAME,
    vector_config=[
        # User-provided embeddings
        Configure.MultiVectors.self_provided(
            name="multi_vector",
        ),
    ],
    properties=[
        Property(name="d_id", data_type=DataType.TEXT),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
    ],
)

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

if client.collections.exists(MUVERA_CLASS_NAME):
    client.collections.delete(MUVERA_CLASS_NAME)

client.collections.create(
    name=MUVERA_CLASS_NAME,
    vector_config=[
        # User-provided embeddings
        Configure.MultiVectors.self_provided(
            name="multi_vector",
            encoding=Configure.VectorIndex.MultiVector.Encoding.muvera(),
        ),
    ],
    properties=[
        Property(name="d_id", data_type=DataType.TEXT),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
    ],
)

In [5]:
colbert_col = client.collections.get(COLBERT_CLASS_NAME)
muvera_col = client.collections.get(MUVERA_CLASS_NAME)

col = muvera_col

In [6]:
import json
import os

dataset = []
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "data", "jqara_test.jsonl")
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)        
        dataset.append(obj)

print(dataset[0])        

{'id': 'QA20CAPR-1004#5191928', 'q_id': 'QA20CAPR-1004', 'passage_row_id': '5191928', 'label': 1, 'text': '絶対零度(ぜったいれいど、アブソリュートゼロ、英: Absolute zero)は、熱力学上の最低温度である摂氏−273.15度。', 'title': '絶対零度 (曖昧さ回避)', 'question': '摂氏ではマイナス273.15度にあたる、全ての原子の振動が停止する最も低い温度を何というでしょう?', 'answers': ['絶対零度']}


In [7]:
import glob
import torch

current_dir = os.getcwd()
batch_paths = sorted(glob.glob(os.path.join(current_dir, "outputs/jacolbert", "batch_*.pt")))

ids = []
embeddings = []  # List[Tensor[n_tokens, dim]]

for p in batch_paths:
    payload = torch.load(p, map_location="cpu")
    ids.extend([str(x) for x in payload.get("ids", [])])
    for e in payload.get("embeddings", []):
        if isinstance(e, torch.Tensor):
            embeddings.append(e.detach().to("cpu").contiguous())
        else:
            embeddings.append(torch.as_tensor(e, dtype=torch.float32))

assert len(ids) == len(embeddings)

print("batches:", len(batch_paths), "docs:", len(embeddings))

batches: 326 docs: 166700


In [8]:
print(dataset[166699])
print(ids[166699])
print(embeddings[166699])

{'id': 'QA20QBIK-2000#4794232', 'q_id': 'QA20QBIK-2000', 'passage_row_id': '4794232', 'label': 0, 'text': 'これは、1973年のビルボード・ホット100による1位のシングル一覧である。背景色を黄色にしているものは、1973年の年間チャートで1位になった楽曲である。', 'title': '1973年のビルボード・ホット100による1位のシングル一覧', 'question': '1977年、アメリカのバンド・イーグルスが放った、砂漠に建つ架空のホテルを舞台にした全米ナンバー1ソングは何?', 'answers': ['ホテル・カリフォルニア']}
QA20QBIK-2000#4794232
tensor([[ 0.1615,  0.1130, -0.1672,  ...,  0.0103, -0.0216, -0.0463],
        [ 0.1477,  0.0896, -0.0544,  ..., -0.0267,  0.1017, -0.0957],
        [ 0.0771,  0.0824, -0.0220,  ..., -0.0178,  0.0833, -0.1333],
        ...,
        [ 0.0711,  0.0994, -0.0850,  ..., -0.0547,  0.0049, -0.0147],
        [ 0.1274,  0.0717, -0.1174,  ..., -0.0082,  0.0231,  0.0080],
        [ 0.0973,  0.0362, -0.0545,  ..., -0.0215,  0.0103,  0.0524]])


In [9]:
from tqdm import tqdm

# N = 166700
N = 66700
subset = dataset[:N]
emb_subset = embeddings[:N]

total = len(subset)

# tqdmで進捗バーを表示
with col.batch.fixed_size(batch_size=16) as batch, tqdm(total=total, desc="📤 Uploading to Weaviate") as pbar:
    for obj, emb in zip(subset, emb_subset):
        # オブジェクトを追加
        batch.add_object(
            properties={
                "d_id": obj["id"],
                "text": obj["text"],
                "title": obj["title"],
            },
            vector={"multi_vector": emb.tolist()},  # ★v4では vectors= を使う
        )

        # 進捗更新
        pbar.update(1)

# 全体完了後のチェック
if col.batch.failed_objects:
    print(f"\n❌ Number of failed imports: {len(col.batch.failed_objects)}")
    print(f"First failed object: {col.batch.failed_objects[0]}")
else:
    print("\n✅ All objects uploaded successfully!")

# コレクション内の件数を確認
count = col.aggregate.over_all(total_count=True).total_count
print(f"🧮 現在の登録件数: {count}")


📤 Uploading to Weaviate: 100%|██████████| 66700/66700 [02:19<00:00, 479.48it/s]



✅ All objects uploaded successfully!
🧮 現在の登録件数: 66700


In [ ]:
from colbert.infra import ColBERTConfig
from colbert.modeling.checkpoint import Checkpoint

ckpt = Checkpoint("jinaai/jina-colbert-v2", colbert_config=ColBERTConfig())

In [11]:

ckpt = Checkpoint("bclavie/JaColBERT", colbert_config=ColBERTConfig())

[Dec 23, 23:11:30] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(


In [12]:
q = subset[600]["question"]

q_vecs = ckpt.queryFromText([q], bsize=1)[0]
print(q_vecs)
print(len(q_vecs))


/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


tensor([[-0.0795,  0.0506, -0.0372,  ...,  0.0853, -0.1266, -0.0209],
        [-0.0320, -0.0024,  0.0659,  ...,  0.0349,  0.0084,  0.0452],
        [ 0.0259, -0.0022,  0.0240,  ...,  0.0968,  0.0656,  0.0342],
        ...,
        [-0.0195, -0.0201,  0.0244,  ..., -0.0054, -0.0255,  0.0256],
        [ 0.0374, -0.0081,  0.0484,  ...,  0.0998, -0.0032,  0.0949],
        [-0.0227,  0.0643,  0.0146,  ...,  0.1327, -0.1094,  0.1478]])
32


In [13]:
from weaviate.classes.query import MetadataQuery


q = subset[0]["question"]
print(q)
q_vecs = ckpt.queryFromText([q], bsize=1)[0]  # [Lq, dim]

response = col.query.near_vector(
    near_vector=q_vecs.tolist(),
    target_vector="multi_vector",
    return_metadata=MetadataQuery(distance=True, certainty=True, score=True),
    limit=1000
)

for o in response.objects:
    title = (o.properties or {}).get("title")
    text = (o.properties or {}).get("text") or ""
    # near_text なら通常 distance / certainty が入る（要求した場合）
    dist = getattr(o.metadata, "distance", None)
    cert = getattr(o.metadata, "certainty", None)
    score = getattr(o.metadata, "score", None)  # ハイブリッド/BM25で付くことがある
    # どれが取得できたかで柔軟に表示
    if dist is not None:
        metric = f"distance={dist:.4f}"
    elif score is not None:
        metric = f"score={score:.4f}"
    elif cert is not None:
        metric = f"certainty={cert:.4f}"
    else:
        metric = "(no metric)"

    print(f"- {metric}  title: {title}")
    print(f"  text: {text[:120]}...")
    print(f"  q_id: {(o.properties or {}).get('q_id')}, answers: {(o.properties or {}).get('answers')}\n")


摂氏ではマイナス273.15度にあたる、全ての原子の振動が停止する最も低い温度を何というでしょう?
- distance=-24.3859  title: 絶対零度
  text: 温度は、物質の熱振動をもとにして規定されているので、下限が存在する。それは、熱振動(原子の振動)が小さくなり、エネルギーが最低になった状態である。この時に決まる下限温度が絶対零度である。古典力学では、エネルギーが最低の状態とは、原子の振動が...
  q_id: None, answers: None

- distance=-24.1298  title: 絶対零度 (曖昧さ回避)
  text: 絶対零度(ぜったいれいど、アブソリュートゼロ、英: Absolute zero)は、熱力学上の最低温度である摂氏−273.15度。...
  q_id: None, answers: None

- distance=-24.1298  title: 絶対零度 (曖昧さ回避)
  text: 絶対零度(ぜったいれいど、アブソリュートゼロ、英: Absolute zero)は、熱力学上の最低温度である摂氏−273.15度。...
  q_id: None, answers: None

- distance=-24.0660  title: 絶対零度
  text: 絶対零度(ぜったいれいど、Absolute zero)は、絶対温度の下限で、理想気体のエントロピーとエンタルピーが最低値になった状態、つまり 0 Kを表す。セルシウス度(摂氏)で −273.15 °C、ファーレンハイト度(華氏)で −459...
  q_id: None, answers: None

- distance=-23.8120  title: 温度
  text: 統計力学によれば、温度は物質を構成する分子の乱雑な並進運動エネルギーの平均値として、五十嵐が導出した様に求めることができる。この様にして求めた温度は、熱力学温度と一致する。熱力学温度の零点(0ケルビン)は絶対零度と呼ばれ、分子の乱雑な並進運...
  q_id: None, answers: None

- distance=-23.7830  title: ボース＝アインシュタイン凝縮
  text: ボース=アインシュタイン凝縮を研究するためには、温

In [14]:
result = col.aggregate.over_all(total_count=True)

print("📊 登録ドキュメント数:", result.total_count)

📊 登録ドキュメント数: 66700


In [15]:
nodes = client.cluster.nodes(output="verbose") # verboseで詳細情報を取得

for node in nodes:
    print(f"Node Name: {node.name}")
    print(f"Status: {node.status}")
    
    # シャードごとの統計情報
    if node.shards:
        for shard in node.shards:
            print(f"  - Class: {shard}")
            print(f"    Object Count: {shard.object_count}")
            # ベクトルインデックスやストレージの情報などを含んでいる場合があります
            # バージョンや設定により取得できるフィールドが異なる可能性があります
    print("-" * 20)



Node Name: node1
Status: HEALTHY
  - Class: Shard(collection='Jqara_colbert', name='9ZBYsRXm7naO', node='node1', object_count=0, vector_indexing_status='READY', vector_queue_length=0, compressed=False, loaded=True)
    Object Count: 0
  - Class: Shard(collection='Jqara', name='hvbyMVyy0J5r', node='node1', object_count=5000, vector_indexing_status='READY', vector_queue_length=0, compressed=False, loaded=True)
    Object Count: 5000
  - Class: Shard(collection='Jqara_muvera', name='YN3StF6xGiI8', node='node1', object_count=65408, vector_indexing_status='READY', vector_queue_length=0, compressed=False, loaded=True)
    Object Count: 65408
  - Class: Shard(collection='Question', name='C4gDZIHzIBb3', node='node1', object_count=10, vector_indexing_status='READY', vector_queue_length=0, compressed=False, loaded=True)
    Object Count: 10
--------------------


In [16]:
from typing import Dict, List, Tuple, Iterable


qid_to_query: Dict[str, str] = {}
qrels: Dict[str, Dict[str, int]] = {}

cnt = 0
for rec in subset:
    cnt += 1
    qid   = rec["q_id"]
    q = rec.get("question")
    docid = rec.get("id")
    rel   = int(rec.get("label", 0))

    if qid not in qid_to_query:
        qid_to_query[qid] = q
    qrels.setdefault(qid, {})[docid] = rel

print(f"Loaded {cnt} records")
print(f"Unique queries: {len(qid_to_query)}")

Loaded 66700 records
Unique queries: 667


In [17]:
from typing import Callable, Awaitable, List, Tuple, Dict
import asyncio

async def build_run_dict_async(
        search_fn: Callable[[str, int], Awaitable[Tuple[List[Tuple[str, float]], float]]],
        qid_to_query: Dict[str, str],
        topk: int = 100
    ) -> Tuple[Dict[str, Dict[str, float]], List[float]]:
    """
    非同期検索関数を使って全クエリを実行し、run_dictとlatenciesを返す。
    
    Args:
        search_fn: async_col が既にバインドされた検索関数 (q, topk) -> ([(doc_id, score), ...], elapsed)
        qid_to_query: {qid: query_text} の辞書
        topk: 各クエリで取得する件数
        
    Returns:
        run_dict: {qid: {doc_id: score, ...}, ...}
        latencies: 各クエリのレイテンシリスト
    """
    CONCURRENCY = 10
    sem = asyncio.Semaphore(CONCURRENCY)    
    run_dict: Dict[str, Dict[str, float]] = {}
    latencies: List[float] = []

    async def _one(qid: str, q: str):
        async with sem:
            ranked, elapsed = await search_fn(q, topk)
            return qid, ranked, elapsed

    tasks = [_one(qid, q) for qid, q in qid_to_query.items()]
    
    for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Searching"):
        qid, ranked, elapsed = await coro
        # ranked は [(doc_id, score), ...] の形式
        run_dict[qid] = {doc_id: score for doc_id, score in ranked}
        latencies.append(elapsed)
    
    return run_dict, latencies

In [18]:

async_client = weaviate.use_async_with_local()
await async_client.connect()

async_col = async_client.collections.get(MUVERA_CLASS_NAME)
q = subset[600]["question"]

q_vecs = ckpt.queryFromText([q], bsize=1)[0]  # [Lq, dim]
print(q)
print(q_vecs)
print(len(q_vecs))

response = async_col.query.near_vector(
    near_vector=q_vecs.tolist(),
    target_vector="multi_vector",
    return_metadata=MetadataQuery(distance=True, certainty=True, score=True)
)

res = await response

print(res)



星条旗といえばアメリカの国旗ですが、太極旗といえばどこの国旗でしょう?
tensor([[-0.0795,  0.0506, -0.0372,  ...,  0.0853, -0.1266, -0.0209],
        [-0.0320, -0.0024,  0.0659,  ...,  0.0349,  0.0084,  0.0452],
        [ 0.0259, -0.0022,  0.0240,  ...,  0.0968,  0.0656,  0.0342],
        ...,
        [-0.0195, -0.0201,  0.0244,  ..., -0.0054, -0.0255,  0.0256],
        [ 0.0374, -0.0081,  0.0484,  ...,  0.0998, -0.0032,  0.0949],
        [-0.0227,  0.0643,  0.0146,  ...,  0.1327, -0.1094,  0.1478]])
32


In [19]:
from typing import List, Tuple
from weaviate.classes.query import MetadataQuery
import time

async def async_muvera_search(async_col, q: str, topk: int = 100) -> Tuple[List[Tuple[str, float]], float]:
    """
    ColBERT (Multi-vector) を用いた非同期検索関数
    
    Args:
        async_col: 非同期Weaviateコレクション
        q: 検索クエリ文字列
        topk: 取得する件数
        
    Returns:
        ([(doc_id, score), ...], elapsed) のタプル
    """
    # クエリのベクトル化 (ColBERT)
    q_vecs_tensor = ckpt.queryFromText([q], bsize=1)[0]
    q_vecs = q_vecs_tensor.tolist()

    # 非同期クエリの実行
    start = time.perf_counter()

    response = await async_col.query.near_vector(
        near_vector=q_vecs,
        target_vector="multi_vector",
        limit=topk,
        return_metadata=MetadataQuery(distance=True)
    )

    elapsed = time.perf_counter() - start

    # 結果の整形: (doc_id, score) のリスト
    results: List[Tuple[str, float]] = []
    for obj in response.objects:
        doc_id = obj.properties.get("d_id", "")
        score = obj.metadata.distance
        results.append((doc_id, score))

    return results, elapsed


def create_muvera_search_fn(async_col):
    """async_colをバインドした検索関数を返すファクトリ"""
    async def search_fn(q: str, topk: int = 100) -> Tuple[List[Tuple[str, float]], float]:
        return await async_muvera_search(async_col, q, topk)
    return search_fn

In [20]:
# async with で接続ライフサイクルを管理
async with weaviate.use_async_with_local() as async_client:
    
    query_text = next(iter(qid_to_query.values()))    
    print(f"Query: {query_text}")
    
    async_col = async_client.collections.get(MUVERA_CLASS_NAME)
    # 関数呼び出し
    hits, elapsed = await async_muvera_search(async_col, query_text, topk=10)
    
    # 結果表示 (hitsは [(doc_id, score), ...] の形式)
    for doc_id, score in hits:
        print(f"Score: {score:.4f} | doc_id: {doc_id}")

    print(f"\nElapsed: {elapsed:.4f} seconds")

Query: 摂氏ではマイナス273.15度にあたる、全ての原子の振動が停止する最も低い温度を何というでしょう?
Score: -24.3859 | doc_id: QA20CAPR-1004#60221
Score: -24.1298 | doc_id: QA20CAPR-1004#5191928
Score: -24.1298 | doc_id: QA20CAPR-1724#5191928
Score: -24.0660 | doc_id: QA20CAPR-1004#60220
Score: -23.8120 | doc_id: QA20CAPR-1004#49522
Score: -23.7830 | doc_id: QA20CAPR-1004#221102
Score: -23.1975 | doc_id: QA20CAPR-1004#60222
Score: -22.9851 | doc_id: QA20CAPR-1004#49531
Score: -22.6809 | doc_id: QA20CAPR-1004#3349535
Score: -22.4734 | doc_id: QA20CAPR-1004#2503329

Elapsed: 0.0382 seconds


/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/weaviate/warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In [21]:
from tqdm import tqdm

# build_run_dict_async を使って全クエリを検索

muvera_search_run_dict = {}
muvera_search_latencies = []

async with weaviate.use_async_with_local() as async_client:
    async_col = async_client.collections.get(MUVERA_CLASS_NAME)
    
    # async_col をバインドした検索関数を作成
    search_fn = create_muvera_search_fn(async_col)
    
    # 全クエリを実行
    muvera_search_run_dict, muvera_search_latencies = await build_run_dict_async(
        search_fn, 
        qid_to_query,
        topk=10
    )

print(f"検索完了: {len(muvera_search_run_dict)} クエリ")
print(f"平均レイテンシ: {sum(muvera_search_latencies) / len(muvera_search_latencies):.4f} 秒")

Searching:   0%|          | 0/667 [00:00<?, ?it/s]/Users/satoshi.takatori.001/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/asyncio/selector_events.py:879: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=110 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
Searching: 100%|██████████| 667/667 [00:17<00:00, 37.56it/s]

検索完了: 667 クエリ
平均レイテンシ: 0.1842 秒


In [22]:
muvera_search_run_dict

{'QA20CAPR-1275': {'QA20CAPR-1275#1243851': -25.883403778076172,
  'QA20CAPR-1275#3889546': -23.762224197387695,
  'QA20CAPR-1235#525160': -23.105606079101562,
  'QA20CAPR-1275#1731240': -22.930784225463867,
  'QA20CAPR-1247#1398498': -22.840221405029297,
  'QA20CAPR-1275#1398498': -22.840221405029297,
  'QA20CAPR-1026#1604302': -22.12588119506836,
  'QA20CAPR-1148#177375': -22.002531051635742,
  'QA20CAPR-1365#1241066': -21.95197296142578,
  'QA20CAPR-1148#1079991': -21.794397354125977},
 'QA20CAPR-1276': {'QA20CAPR-1120#2256469': -21.655689239501953,
  'QA20CAPR-1942#22134': -20.704269409179688,
  'QA20CAPR-1414#67706': -20.614261627197266,
  'QA20CAPR-1771#2325165': -20.307748794555664,
  'QA20CAPR-1111#890297': -20.2938175201416,
  'QA20CAPR-1675#171809': -20.180849075317383,
  'QA20CAPR-1675#65871': -20.07039451599121,
  'QA20CAPR-1540#65871': -20.07039451599121,
  'QA20CAPR-1654#4773501': -19.908058166503906,
  'QA20CAPR-1771#3749800': -19.85711097717285},
 'QA20CAPR-1800': {'QA2

In [23]:
from ranx import Qrels, Run, evaluate

def evaluate_run_dict(run_dict: Dict[str, Dict[str, float]], qrels: Dict[str, Dict[str, int]]):
    qrels_ranx = Qrels(qrels)
    run_ranx   = Run(run_dict)

    metrics = [
        "ndcg@1", "ndcg@3", "ndcg@5", "ndcg@10",
        "map", "mrr", "precision@10", "recall@10", "recall@100",
    ]

    scores = evaluate(qrels_ranx, run_ranx, metrics=metrics)
    for m in metrics:
        print(f"{m:>12}: {scores[m]:.4f}")

In [24]:
import numpy as np

def print_latency_stats(latencies: List[float]):
    l = np.array(latencies)

    print(f"Count    : {len(l)}")
    print(f"Mean     : {l.mean():.4f} sec")
    print(f"Median   : {np.median(l):.4f} sec")
    print(f"p90      : {np.percentile(l,90):.4f} sec")
    print(f"p95      : {np.percentile(l,95):.4f} sec")
    print(f"p99      : {np.percentile(l,99):.4f} sec")
    print(f"Min/Max  : {l.min():.4f} / {l.max():.4f} sec")

In [25]:
evaluate_run_dict(muvera_search_run_dict, qrels)
print_latency_stats(muvera_search_latencies)

      ndcg@1: 0.1769
      ndcg@3: 0.1925
      ndcg@5: 0.2039
     ndcg@10: 0.3109
         map: 0.1276
         mrr: 0.3479
precision@10: 0.3082
   recall@10: 0.3398
  recall@100: 0.3398
Count    : 667
Mean     : 0.1842 sec
Median   : 0.1819 sec
p90      : 0.2376 sec
p95      : 0.2434 sec
p99      : 0.2949 sec
Min/Max  : 0.0262 / 0.5905 sec
